Setting Up

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.4f}'.format
 
import warnings
warnings.filterwarnings('ignore') 
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=1gZnD5nCVL3_HBlzo63Q-fS46qAZfJDVt'
link2= 'https://drive.google.com/open?id=10_xXrkGExszPjGDGeefVa8bQtuTsKJqT'
link3= 'https://drive.google.com/open?id=1seJJJ4qXGHQIC64P_A9MbBCYLvngd9Bd'

fluff, id = link.split('=')
fluff2, id2 = link2.split('=')
fluff3, id3 = link3.split('=')

print (id) # Verify that you have everything after '='
print (id2) # Verify that you have everything after '='
print (id3) # Verify that you have everything after '='
 
 
downloaded = drive.CreateFile({'id':id})
downloaded2 = drive.CreateFile({'id':id2})
downloaded3 = drive.CreateFile({'id':id3}) 

downloaded.GetContentFile('Train.csv')
downloaded2.GetContentFile('Test.csv')
downloaded3.GetContentFile('SampleSubmission.csv')  
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')
# Dataset is now stored in a Pandas Datafra

1gZnD5nCVL3_HBlzo63Q-fS46qAZfJDVt
10_xXrkGExszPjGDGeefVa8bQtuTsKJqT
1seJJJ4qXGHQIC64P_A9MbBCYLvngd9Bd


In [ ]:
df_train.head(2)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0000,15.0000,4251.0000,1417.0000,17.0000,4.0000,388.0000,46.0000,1.0000,1.0000,2.0000,NO,54,On net 200F=Unlimited _call24H,8.0000,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NO,4,NaN,nan,1


**Handling missing Values(Continious)**

 

In [ ]:
#View numericals
num =df_train._get_numeric_data().columns
num

Index(['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE',
       'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2',
       'REGULARITY', 'FREQ_TOP_PACK', 'CHURN'],
      dtype='object')

In [ ]:
#Dropping UserID and MRG
df_train.drop(['MRG','user_id'],1,inplace=True)

df_test.drop(['MRG','user_id'],1,inplace=True)

In [ ]:
#Capturing importance of missing values in colomns
def importance(df,var):
  df[var + '_NAN'] = np.where(df[var].isnull(),1,0)
 
for feature in ['MONTANT','ORANGE','FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE','DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2','REGULARITY', 'FREQ_TOP_PACK']:
  importance(df_train, feature)
  importance(df_test, feature)

In [ ]:
#NAN Imputers
def imputer(df,variable):
  df[variable] = df[variable].fillna(df[variable].median())
  
for feature in ['MONTANT','ORANGE','FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE','DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2','REGULARITY', 'FREQ_TOP_PACK']:
  imputer(df_train,feature)
  imputer(df_test,feature)

In [ ]:
df_train.head(2)#Great we done with numericals

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN,MONTANT_NAN,ORANGE_NAN,FREQUENCE_RECH_NAN,REVENUE_NAN,ARPU_SEGMENT_NAN,FREQUENCE_NAN,DATA_VOLUME_NAN,ON_NET_NAN,TIGO_NAN,ZONE1_NAN,ZONE2_NAN,REGULARITY_NAN,FREQ_TOP_PACK_NAN
0,FATICK,K > 24 month,4250.0000,15.0000,4251.0000,1417.0000,17.0000,4.0000,388.0000,46.0000,1.0000,1.0000,2.0000,54,On net 200F=Unlimited _call24H,8.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,I 18-21 month,3000.0000,7.0000,3000.0000,1000.0000,9.0000,257.0000,27.0000,29.0000,6.0000,1.0000,2.0000,4,NaN,5.0000,1,1,1,1,1,1,1,1,1,1,1,1,0,1


****Taking care of Cats(categorical values)****

In [ ]:
###Due to high cardinality in this feature, we'll go with The top ten cats
df_train.TOP_PACK.value_counts().sort_values(ascending=False).head(10)

All-net 500F=2000F;5d                        317802
On net 200F=Unlimited _call24H               152295
Data:490F=1GB,7d                             115180
Data: 100 F=40MB,24H                          84649
Mixt 250F=Unlimited_call24H                   67512
MIXT:500F= 2500F on net _2500F off net;2d     64412
Data:1000F=2GB,30d                            59770
All-net 500F =2000F_AllNet_Unlimited          46890
Jokko_Daily                                   45036
Data: 200 F=100MB,24H                         42841
Name: TOP_PACK, dtype: int64

In [ ]:
top10 = df_train.TOP_PACK.value_counts().sort_values(ascending=False).head(10).index
top10=list(top10) 
top10

['All-net 500F=2000F;5d',
 'On net 200F=Unlimited _call24H',
 'Data:490F=1GB,7d',
 'Data: 100 F=40MB,24H',
 'Mixt 250F=Unlimited_call24H',
 'MIXT:500F= 2500F on net _2500F off net;2d',
 'Data:1000F=2GB,30d',
 'All-net 500F =2000F_AllNet_Unlimited',
 'Jokko_Daily',
 'Data: 200 F=100MB,24H']

In [ ]:
for categories in top10:
    df_train[categories]=np.where(df_train['TOP_PACK']==categories,1,0)
    df_test[categories]=np.where(df_test['TOP_PACK']==categories,1,0)
 
df_train.drop(['TOP_PACK'],1,inplace=True)
df_test.drop(['TOP_PACK'],1,inplace=True)
df_train.head(2)

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK,CHURN,MONTANT_NAN,ORANGE_NAN,FREQUENCE_RECH_NAN,REVENUE_NAN,ARPU_SEGMENT_NAN,FREQUENCE_NAN,DATA_VOLUME_NAN,ON_NET_NAN,TIGO_NAN,ZONE1_NAN,ZONE2_NAN,REGULARITY_NAN,FREQ_TOP_PACK_NAN,All-net 500F=2000F;5d,On net 200F=Unlimited _call24H,"Data:490F=1GB,7d","Data: 100 F=40MB,24H",Mixt 250F=Unlimited_call24H,MIXT:500F= 2500F on net _2500F off net;2d,"Data:1000F=2GB,30d",All-net 500F =2000F_AllNet_Unlimited,Jokko_Daily,"Data: 200 F=100MB,24H"
0,FATICK,K > 24 month,4250.0000,15.0000,4251.0000,1417.0000,17.0000,4.0000,388.0000,46.0000,1.0000,1.0000,2.0000,54,8.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,NaN,I 18-21 month,3000.0000,7.0000,3000.0000,1000.0000,9.0000,257.0000,27.0000,29.0000,6.0000,1.0000,2.0000,4,5.0000,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
##Ordinal Encode TENURE
 
df_train['TENURE'] = df_train['TENURE'].map({
    'K > 24 month':8,
    'I 18-21 month':6,
    'G 12-15 month':4,
    'H 15-18 month':5,
    'J 21-24 month':7,
    'F 9-12 month':3,
    'D 3-6 month':1,
    'E 6-9 month':2,    
}) 
 
 
 
df_test['TENURE'] = df_test['TENURE'].map({
    'K > 24 month':8,
    'I 18-21 month':6,
    'G 12-15 month':4,
    'H 15-18 month':5,
    'J 21-24 month':7,
    'F 9-12 month':3,
    'D 3-6 month':1,
    'E 6-9 month':2,    
})  
 
df_train.TENURE.unique()

array([8, 6, 4, 5, 7, 3, 1, 2])

In [ ]:
#CApture importance of missing values in REGION 
df_train['REGION_NAN'] = np.where(df_train['REGION'].isnull(),1,0)
df_test['REGION_NAN'] = np.where(df_test['REGION'].isnull(),1,0) 
##Replace NAN in REGION with OTHER
df_train.REGION.fillna('OTHER',inplace=True)
df_test.REGION.fillna('OTHER',inplace=True)

In [ ]:
mean_ordinal=df_train.groupby(['REGION'])['CHURN'].mean().sort_values().to_dict()
mean_ordinal

{'DAKAR': 0.019235452616648906,
 'DIOURBEL': 0.028097024405553644,
 'FATICK': 0.014196335886429313,
 'KAFFRINE': 0.007187862520756091,
 'KAOLACK': 0.023436372259913803,
 'KEDOUGOU': 0.04215686274509804,
 'KOLDA': 0.010763234648839791,
 'LOUGA': 0.01595105650510333,
 'MATAM': 0.02008045937489255,
 'OTHER': 0.4479871046592543,
 'SAINT-LOUIS': 0.012678711442537077,
 'SEDHIOU': 0.043603719140750244,
 'TAMBACOUNDA': 0.015361150452118968,
 'THIES': 0.016300846422144712,
 'ZIGUINCHOR': 0.027432216905901117}

In [ ]:
#Mapping mean ordinals to REGION
df_train.REGION=df_train.REGION.map(mean_ordinal)
df_test.REGION=df_test.REGION.map(mean_ordinal)
df_train.head(2)

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK,CHURN,MONTANT_NAN,ORANGE_NAN,FREQUENCE_RECH_NAN,REVENUE_NAN,ARPU_SEGMENT_NAN,FREQUENCE_NAN,DATA_VOLUME_NAN,ON_NET_NAN,TIGO_NAN,ZONE1_NAN,ZONE2_NAN,REGULARITY_NAN,FREQ_TOP_PACK_NAN,All-net 500F=2000F;5d,On net 200F=Unlimited _call24H,"Data:490F=1GB,7d","Data: 100 F=40MB,24H",Mixt 250F=Unlimited_call24H,MIXT:500F= 2500F on net _2500F off net;2d,"Data:1000F=2GB,30d",All-net 500F =2000F_AllNet_Unlimited,Jokko_Daily,"Data: 200 F=100MB,24H",REGION_NAN
0,0.0142,8,4250.0000,15.0000,4251.0000,1417.0000,17.0000,4.0000,388.0000,46.0000,1.0000,1.0000,2.0000,54,8.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0.4480,6,3000.0000,7.0000,3000.0000,1000.0000,9.0000,257.0000,27.0000,29.0000,6.0000,1.0000,2.0000,4,5.0000,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
df_test.head(1)

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK,MONTANT_NAN,ORANGE_NAN,FREQUENCE_RECH_NAN,REVENUE_NAN,ARPU_SEGMENT_NAN,FREQUENCE_NAN,DATA_VOLUME_NAN,ON_NET_NAN,TIGO_NAN,ZONE1_NAN,ZONE2_NAN,REGULARITY_NAN,FREQ_TOP_PACK_NAN,All-net 500F=2000F;5d,On net 200F=Unlimited _call24H,"Data:490F=1GB,7d","Data: 100 F=40MB,24H",Mixt 250F=Unlimited_call24H,MIXT:500F= 2500F on net _2500F off net;2d,"Data:1000F=2GB,30d",All-net 500F =2000F_AllNet_Unlimited,Jokko_Daily,"Data: 200 F=100MB,24H",REGION_NAN
0,0.0163,8,5000.0000,5.0000,5000.0000,1667.0000,5.0000,259.0000,378.0000,11.0000,5.0000,1.0000,2.0000,42,5.0000,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
###DONE WITH CLEANING###

Baseline Models

In [ ]:
#assigning x and y
x_train= df_train.drop('CHURN',1)
y_train= df_train.CHURN

In [ ]:
pip install xgboost

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

#Baseline Random forest
rf= RandomForestClassifier()
baseline_rf= rf.fit(x_train,y_train)

#Baseline XGBOOSTCLASSIER
xg = XGBClassifier()
baseline_xg= xg.fit(x_train,y_train)

#Baseline LGBMClassifier
lg = lgb.LGBMClassifier()
baseline_lg= lg.fit(x_train,y_train)

In [ ]:
###Here are the predictions
y_pred1= baseline_rf.predict_proba(df_test)[:,1]
y_pred2= baseline_xg.predict_proba(df_test)[:,1]
y_pred3= baseline_lg.predict_proba(df_test)[:,1]

#SUBMISSION
 
ans= pd.DataFrame(y_pred1)
df_sub=pd.read_csv('SampleSubmission.csv')
df_sub= df_sub.drop('CHURN',1)
sub = pd.concat([df_sub['user_id'],ans],1)
sub.columns=['user_id','target']
sub.to_csv('baseline_rf.csv',index=False)

ans= pd.DataFrame(y_pred2)
df_sub=pd.read_csv('SampleSubmission.csv')
df_sub= df_sub.drop('CHURN',1)
sub = pd.concat([df_sub['user_id'],ans],1)
sub.columns=['user_id','target']
sub.to_csv('baseline_xg.csv',index=False)

ans= pd.DataFrame(y_pred3)
df_sub=pd.read_csv('SampleSubmission.csv')
df_sub= df_sub.drop('CHURN',1)
sub = pd.concat([df_sub['user_id'],ans],1)
sub.columns=['user_id','target']
sub.to_csv('baseline_lg.csv',index=False)

